In [1]:
import os

os.chdir('/home/ubuntu/mlops_zoomcamp/mlops-zoomcamp-2024/06-best-practices')

In [2]:
import pickle
from tqdm import tqdm
import pandas as pd
import polars as pl

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import root_mean_squared_error

In [4]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("yellow-taxi-duration")

<Experiment: artifact_location='s3://saiful-mflow-artifact-store/1', creation_time=1719744296684, experiment_id='1', last_update_time=1719744296684, lifecycle_stage='active', name='yellow-taxi-duration', tags={}>

In [4]:
import requests

year = 2024
start_month = 1
end_month = 3

for month in range(start_month, end_month + 1):
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet"
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        save_path=f"data/yellow_tripdata_{year:04d}-{month:02d}.parquet"
        with open(save_path, "wb") as handle:
            for data in tqdm(response.iter_content(),
                             desc=f"yellow_tripdata_{year:04d}-{month:02d}.parquet",
                             postfix=f"save to {save_path}",
                             total=int(response.headers["Content-Length"])
            ):
                handle.write(data)
    else:
        print(f"Failed to download data for {year:04d}-{month:02d}")

yellow_tripdata_2024-03.parquet: 100%|██████████| 60078280/60078280 [06:05<00:00, 164262.45it/s, save to data/yellow_tripdata_2024-03.parquet]


In [6]:
def read_dataframe(filename: str) -> pd.DataFrame:
    df = pl.scan_parquet(filename).with_columns(
        duration = (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")).dt.total_seconds() / 60
    ).filter(
        (pl.col("duration") > 0) & (pl.col("duration") < 60)
    ).with_columns(
        pl.col('PULocationID').cast(pl.Utf8).alias('PULocationID'),
        pl.col('DOLocationID').cast(pl.Utf8).alias('DOLocationID'),
    ).select(
        ['trip_distance', 'PULocationID', 'DOLocationID', 'duration']
    )

    return df.collect().to_pandas()

def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
df_train = read_dataframe('data/yellow_tripdata_2024-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2024-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [9]:
len(dict_train)

2933099

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
with mlflow.start_run():
    params = dict(
        max_depth=20,
        n_estimators=100,
        min_samples_leaf=10,
        random_state=42
    )
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric("rmse", rmse)

    mlflow.sklearn.log_model(pipeline, "model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 42} 5.4584915452858285
